- references
    - https://docs.scipy.org/doc/scipy/tutorial/optimize.html

## 参数

### method

- SLSQP
- BFGS

### bounds & constraints

- bounds：决策变量的最小最大；
    - `(0, None)`: `x >= 0`
- constraints
    - type: `ineq`, `>= 0`
    - type: `eq`, `=0`

## L-BFGS-B

$$
\begin{split}
&s_k=x_{k+1}-x_k\\
&y_k=\nabla f_{k+1}-\nabla f_k\\
&H_{k+1}=\left(I-\rho_ks_ky_k^T\right)H_k\left(I-\rho_ks_ky_k^T\right)+\rho_ks_ks_k^T, \quad \rho_k=\frac{1}{y_k^Ts_k}
\end{split}
$$

L-BFGS-B算法是一种优化算法，用于解决**有界约束**的优化问题。它是限制性Broyden-Fletcher-Goldfarb-Shanno（BFGS）算法的变种，专门设计来处理变量的边界约束问题。L-BFGS-B算法在机器学习和统计建模中广泛应用，特别是在最大似然估计（MLE）中优化参数时。

算法原理
L-BFGS-B算法基于几个关键概念：

- 拟牛顿方法（Quasi-Newton Methods）：这些方法是用来求解无约束优化问题的，它们通过构造目标函数的海森矩阵（Hessian Matrix）的近似来迭代寻找最优解。
- BFGS：是拟牛顿方法中的一种，通过迭代更新逆海森矩阵（$H_k$）的估计来优化目标函数。它不需要计算真正的海森矩阵，而是利用梯度信息来构造近似。
- 有限内存（Limited Memory）：L-BFGS通过只保留最近几步的更新信息来减少存储需求，使得算法能够处理大规模优化问题。
- 边界约束（Bound Constraints）：L-BFGS-B扩展了L-BFGS算法，允许对参数施加上下界约束。这通过特别的技术处理被约束的变量来实现，保证算法迭代不会超出指定的边界。

算法步骤
L-BFGS-B算法的实现步骤大致如下：

- 初始化：选择初始参数值，设置边界约束，确定内存限制（即保留多少步的历史信息）。
- 迭代优化：
    - 计算当前参数下的目标函数值和梯度。
    - 使用有限内存BFGS方法更新逆海森矩阵的近似。
    - 根据更新的逆海森矩阵和梯度，通过求解一个有界约束的二次近似问题来确定搜索方向和步长。
    - 检查边界约束，调整步长以确保新的参数值在约束内。
    - 更新参数值。
- 终止条件：当满足一定的终止条件（如梯度的范数小于某个阈值，或达到最大迭代次数）时，停止迭代。


## 一个示例（with callback）

In [3]:
from scipy import optimize as op
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

x_iter = []

def callback(xk):
    x_iter.append(xk.copy())

x0 = np.array([10., 10])
rmax = 2.
r =.25
eta = 1./16
tol = 1e-5
opts = {'initial_trust_radius': r, 
        'max_trust_radius': 
        rmax, 'eta': eta, 
        'gtol': tol, 
        'disp': True}
sol1 = op.minimize(op.rosen, x0, method='dogleg', jac=op.rosen_der, hess=op.rosen_hess, options=opts, callback=callback)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 32
         Function evaluations: 33
         Gradient evaluations: 31
         Hessian evaluations: 30


In [4]:
sol1

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 2.7861938468014046e-18
       x: [ 1.000e+00  1.000e+00]
     nit: 32
     jac: [ 1.051e-08 -3.598e-09]
    nfev: 33
    njev: 31
    nhev: 30
    hess: [[ 8.020e+02 -4.000e+02]
           [-4.000e+02  2.000e+02]]